In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os


os.makedirs("processed_images", exist_ok=True)

# Load the image
image_path = "clean.jpg"
image = cv2.imread(image_path)
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 1. Pixel-Level Analysis: Calculate Mean Intensity and Histogram
mean_intensity = np.mean(gray_image)
print(f"Mean Pixel Intensity: {mean_intensity:.2f}")

# Display Histogram
plt.hist(gray_image.ravel(), bins=256, range=[0, 256])  # Updated to fix deprecation warning
plt.title('Histogram of Pixel Intensities')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')
plt.savefig('processed_images/pixel_histogram.png')
plt.close()

# 2. Image Processing Techniques
# Grayscale to RGB Conversion (for visualization)
rgb_image = cv2.cvtColor(gray_image, cv2.COLOR_GRAY2BGR)

# Edge Detection using Canny
edges = cv2.Canny(gray_image, 100, 200)

# Thresholding for Defect Detection
_, thresholded = cv2.threshold(gray_image, 120, 255, cv2.THRESH_BINARY_INV)

# Find Contours (defective areas)
contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Calculate Defect Area
total_defect_area = sum([cv2.contourArea(contour) for contour in contours if cv2.contourArea(contour) > 100])

# Calculate Total Image Area
total_image_area = gray_image.shape[0] * gray_image.shape[1]

# Defect Percentage
defect_percentage = (total_defect_area / total_image_area) * 100

# Annotate Detected Defects
for contour in contours:
    if cv2.contourArea(contour) > 100:  # Filter small noise
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)  # Draw bounding box around defect

# Print Defect Prediction Output
if defect_percentage == 0:
    print("Defect Prediction: No defects detected (0%)")
elif defect_percentage < 5:
    print(f"Defect Prediction: Minor defects detected ({defect_percentage:.2f}%)")
elif defect_percentage < 20:
    print(f"Defect Prediction: Moderate defects detected ({defect_percentage:.2f}%)")
else:
    print(f"Defect Prediction: Severe defects detected ({defect_percentage:.2f}%)")

# 3. Image Format Handling and Compression
cv2.imwrite("processed_images/defect_detected.png", image)  # Save as PNG
cv2.imwrite("processed_images/defect_detected_compressed.jpg", image, [cv2.IMWRITE_JPEG_QUALITY, 50])  # Compressed JPEG
cv2.imwrite("processed_images/defect_detected.tiff", image)  # Save as TIFF

print("Image processing complete. Processed images saved in 'processed_images' folder.")


Mean Pixel Intensity: 129.05
Defect Prediction: Severe defects detected (34.87%)
Image processing complete. Processed images saved in 'processed_images' folder.
